In [1]:

from scipy.linalg import norm
from scipy import sum, average

from __future__ import division

import cv2
from os import path
from glob import glob 

In [8]:
def compare_images(img1, img2, verbose=False):
    
    # convert image to gray scale
    img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    
    # crop images to required size
    img1 = crop_resize(img1)
    img2 = crop_resize(img2)
    
    # normalize images
    img1 = normalize(img1)
    img2 = normalize(img2)
    
    # calculate the difference and its norms
    diff = img1 - img2  # elementwise for scipy arrays
    m_norm = sum(abs(diff))  # Manhattan norm
    z_norm = norm(diff.ravel(), 0)  # Zero norm
    if verbose:
        print "Manhattan norm per pixel:", m_norm/img1.size
    
    return (m_norm/img1.size)

In [2]:
def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)*255/rng

In [3]:
def crop_resize(img, height=480, width=640):
    
    imgH = img.shape[0]
    imgW = img.shape[1]
    
    ratio = height / width
    img_ratio = imgH / imgW
    
    # adjust height to width ratio to required size
    if img_ratio > ratio:
        c_img = img[0:int(ratio * imgW), 0:imgW]
    elif img_ratio < ratio:
        c_img = img[0:imgH, 0:(imgH / ratio)]
    else:
        c_img = img[0:imgH, 0:imgW]
     
    c_img = cv2.resize(c_img, (height, width)) # adjust resolution
    return c_img

In [5]:
iris = cv2.imread('S3001L01.jpg')

fb1 = cv2.imread('18138004_163325265000_2.jpg')
fb2 = cv2.imread('20130711120117-1161777702.jpg')
fb3 = cv2.imread('9667726_082238294187_2.jpg')

In [26]:
# read ten iris images as sample images
def get_samples(dr="sample_images/", ext="jpg"):
    images = glob(path.join(dr,"*.{}".format(ext)))
    for i in range(len(images)):
        img = images[i]
        img = cv2.imread(img)
        images[i] = img
    return images

In [32]:
samples = get_samples()
dists = []
for sam in samples:
    dists.append(compare_images(sam, fb3))
print average(dists)

0.128886025471


In [33]:
# check if uploaded image similar to sample iris images
def judge_similarity(img, threshold = 0.18, verbose=False):
    samples = get_samples()
    dists =  []
    for sam in samples:
        dists.append(compare_images(sam, img))
    distance = average(dists)
    if verbose:
        print 'Threshold value is:', threshold
        print 'Distance between image and sample images is:', distance
    return distance <= threshold

In [36]:
judge_similarity(fb1, verbose=True)

Threshold value is: 0.18
Distance between image and sample images is: 0.228848837636


C:\Users\THINKPAD\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


False